<a href="https://colab.research.google.com/github/robfalck/dymos_tutorial/blob/main/01_dymos_simple_driver_boundary_value_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dymos: Using an Optimizer to Solve a Simple Boundary Value Problem

In the previous notebook, we demonstrated
- how to install Dymos
- how to define a simple ODE system
- how to use that system to propagate a simple trajectory

Using the `scipy.itegrate.solve_ivp` functionality from within Dymos will provide a trajectory assuming we know the initial state, control profile, and time duration of that trajectory.

In our next use case we want to solve a simple boundary value problem:

_How far will the cannonball fly?_

In order to determine this, we need to allow the duration of the flight to vary such that the final height of the cannonball is zero.  There are two general approaches to this.

1. Use an optimizer and pose the problem as an optimal control problem.
2. Use a nonlinear solver and pose the problem with residuals to be driven to zero.

# Posing the cannonball boundary value problem as an optimal control problem.

The optimal control problem can be stated as:

\begin{align}
  \mathrm{Minimize}\;J &= t_f \\
  \mathrm{subject\;to\!:} \\
  x(t_0) &= 0 \;\mathrm{m} \\
  y(t_0) &= 0 \;\mathrm{m}\\
  vx(t_0) &= 100 \;\mathrm{m/s} \\
  vy(t_0) &= 100 \;\mathrm{m/s} \\
  y(t_f) &= 0 \;\mathrm{m}
\end{align}

Traditionally, the collocation techniques in Dymos have used an optimizer to satisfy the _defect constraints_ ($\Delta$).  
These defects are the difference between the slope of each state polynomial representation in each segment and the rate of change as given by the ODE.
In Dymos, these defects are measured at a certain subset of our nodes called the _collocation nodes_.  For the Radau pseudospectral transcription, each 3rd-order segment has 4 state input nodes, and 3 collocation nodes.
Dymos handles this underlying transcription from an optimal control problem into a nonlinear programming (NLP) problem.
The equivalent NLP problem for our optimal control problem above is:

\begin{align}
  \mathrm{Minimize}\;J &= t[-1] \\
  \mathrm{subject\;to\!:} \\
  x_{lb} &\le x_i \le x_{ub} \\
  y_{lb} &\le y_i \le y_{ub} \\
  vx_{lb} &\le vx_i \le vx_{ub} \\
  vy_{lb} &\le vy_i \le vy_{ub} \\
  \Delta x_j &= 0 \\
  \Delta y_j &= 0 \\
  \Delta vx_j &= 0 \\
  \Delta vy_j &= 0 \\
  vy[-1] &= 0 \\
  i &= 1\;...n-1 \\
  j &= 0\;...n-2 \\
\end{align}

In the case of our single 3rd-order Radau segment, we have 4 nodes (n=4).
Since each initial state is fixed, we have 12 design variables and 12 constraints.  There is, at most, a single solution to this problem.  This makes sense.  Since there are no control variables or changes in the initial state allowed, there should be only one possible physical path for the cannonball.  Most optimizers will be perfectly capable of solving this problem, despite the fact that it only has a single feasible solution.

# Solving the optimal control problem

First, lets bring in our definition of the ODE and the other general setup that we used in the previous notebook.

In [6]:
!pip install dymos

In [7]:
import numpy as np
import openmdao.api as om
import dymos as dm

In [8]:
class ProjectileODE(om.ExplicitComponent):

  def initialize(self):
    self.options.declare('num_nodes', types=int,
                         desc='the number of points at which this ODE is simultaneously evaluated')

  def setup(self):
    nn = self.options['num_nodes']
    self.add_input('vx', shape=(nn,), units='m/s')
    self.add_input('vy', shape=(nn,), units='m/s')

    self.add_output('x_dot', shape=(nn,), units='m/s',
                    tags=['state_rate_source:x', 'state_units:m'])

    self.add_output('y_dot', shape=(nn,), units='m/s',
                    tags=['state_rate_source:y', 'state_units:m'])

    self.add_output('vx_dot', shape=(nn,), units='m/s**2',
                    tags=['state_rate_source:vx', 'state_units:m/s'])

    self.add_output('vy_dot', shape=(nn,), units='m/s**2',
                    tags=['state_rate_source:vy', 'state_units:m/s'])
    
    self.declare_partials(of='*', wrt='*', method='fd')
    
  def compute(self, inputs, outputs):
    outputs['x_dot'][:] = inputs['vx']
    outputs['y_dot'][:] = inputs['vy']
    outputs['vx_dot'][:] = 0.0
    outputs['vy_dot'][:] = -9.81
    


In [9]:
prob = om.Problem()

traj = prob.model.add_subsystem('traj', dm.Trajectory())

phase = traj.add_phase('phase', dm.Phase(ode_class=ProjectileODE, transcription=dm.Radau(num_segments=1, order=3)))

phase.set_time_options(fix_initial=True, duration_bounds=(1, 1000), units='s')

phase.set_state_options('x', fix_initial=True)
phase.set_state_options('y', fix_initial=True)
phase.set_state_options('vx', fix_initial=True)
phase.set_state_options('vy', fix_initial=True)

# Imposing state boundary constraints

There are two ways in which we can impose boundary constraints on the state.  The first is to use the arguments `fix_initial` and `fix_final`.

Using `fix_final` removes the final value of the state as a design variable.  That is, our final value of `y` will always be satisfied and the optimizer will work to find the rest of the state history.

The other option is to use a nonlinear boundary constraint.  This is a constraint applied after-the-fact.  The optimizer is free to suggest a final value of `y` that violates the constraint, but then will work to satisfy it as it iterates.  This approach gives the optimizer a bit more freedom to explore the design space and might therefore be more robust.  Using `fix_initial` and `fix_final` also impose limitations when using shooting methods in Dymos, which we'll discuss later.

In [10]:
phase.add_boundary_constraint('y', loc='final', equals=0)

# Adding an objective

Using an optimization driver **requires** that an objective be specified.
Again, in this case it's somewhat meaningless, there should only be a single valid solution that satisfies all of our constraints.
To satisfy the driver, we'll set the objective to be the final time.
Note that the Dymos `Phase` object has an `add_objective` method that's similar to the standard OpenMDAO `add_objective` method, but also allows us to specify the _locaiton_ in time where the objective should be evaluated (`'initial'` or `'final'`).

The variable `'time'` is recognized by Dymos as the one of the special variables in the phase (along with states, controls, and parameters).  If we wanted to, we could also make any output of the ODE an objective by specifying the ODE-relative path to that variable.

In [17]:
phase.add_objective('time', loc='final', ref=1.0)

# Specifying a driver

Since we're performing optimization now, we'll need a driver.
The optimizers in `scipy.optimize` will work in this case.  To use them, we use OpenMDAO's `ScipyOptimizeDriver`.

In [18]:
prob.driver = om.ScipyOptimizeDriver()

Next we call OpenMDAO's `Problem.setup` method.  This works out the various connections between the different systems that Dymos uses to solve this problem and allocates memory.  You can think of this step a bit like compiling our model before we run it.

In [19]:
prob.setup()

As before, we'll specify an initial guess for the initial time and duration of the phase, along with linear guesses to the state-time history.

In [20]:
prob.set_val('traj.phase.t_initial', 0.0, units='s')
prob.set_val('traj.phase.t_duration', 15.0, units='s')

prob.set_val('traj.phase.states:x', phase.interpolate(ys=[0, 100], nodes='state_input'), units='m')
prob.set_val('traj.phase.states:y', phase.interpolate(ys=[0, 0], nodes='state_input'), units='m')
prob.set_val('traj.phase.states:vx', phase.interpolate(ys=[100, 100], nodes='state_input'), units='m/s')
prob.set_val('traj.phase.states:vy', phase.interpolate(ys=[100, -100], nodes='state_input'), units='m/s')

# Solving the Problem

To solve this problem, we need to allow the driver to iterate.
The native OpenMDAO option to do so is `Problem.run_driver`.

Dymos provides its own function, `dymos.run_problem`, which is basically a wrapper around the OpenMDAO method with some additional conveniences built in.
Though not necessary yet, we'll use that method just to get in the habit of doing so.


In [21]:
dm.run_problem(prob, run_driver=True, simulate=True, make_plots=True)

Model viewer data has already has already been recorded for Driver.
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 20.387359836901105
            Iterations: 2
            Function evaluations: 2
            Gradient evaluations: 2
Optimization Complete
-----------------------------------

Simulating trajectory traj


/usr/local/lib/python3.6/dist-packages/openmdao/recorders/recording_manager.py:266: UserWarning:The model is being run again, if the options or scaling of any components has changed then only their new values will be recorded.


Done simulating trajectory traj


In [22]:
# TODO: Make these more automatic (and use an interactive plot library like bokeh)

from IPython.display import display
from ipywidgets import widgets, GridBox

items = [widgets.Image(value=open(f'plots/states_{state}.png', 'rb').read()) for state in ['x', 'y', 'vx', 'vy']] + \
[widgets.Image(value=open(f'plots/state_rates_{state}.png', 'rb').read()) for state in ['x', 'y', 'vx', 'vy']]
widgets.GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(2, 500px)"))


GridBox(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\xb0\x00\x00\x01 \x08\x06\x00\x…

# The Solution

The plots above show that the implicit solution and the simulated trajectory are now in agreement (the simulated trajectory is a reasonably accurate interpolation of the solution).

To extract the solution, we can pull values for the special variables in the phase from the _timeseries_ output.

In [25]:
t = prob.get_val('traj.phase.timeseries.time')
x = prob.get_val('traj.phase.timeseries.states:x')
y = prob.get_val('traj.phase.timeseries.states:y')

print(f'The time of flight is {t[-1, 0]} sec')
print(f'The range flown is {x[-1, 0]} m')
print(f'The final altitude of the cannonball is {y[-1, 0]} m')

The time of flight is 20.387359836901105 sec
The range flown is 2038.7359836901096 m
The final altitude of the cannonball is -5.684341886080802e-14 m
